### Welcome

This is the first section of the introductory GeoIPS tutorial, which includes running GeoIPS 
using the CLI, and creating your first plugin! 


Link to these slides 
https://github.com/NRLMMD-GEOIPS/presentations

### Tutorial Scope

This tutorial does not address running GeoIPS in near real-time. GeoIPS plugins 
are intended to be developed and tested on a specific dataset, and setting up the 
real-time processing infrastructure is a separate conversation. 

This tutorial focuses on: 

  - Installing GeoIPS 
  - Running GeoIPS 
  - Product development and testing 


### To get the most out of this tutorial… 

- [Aug 2024 - Intro GeoIPS Tutorial - Google Slides](https://docs.google.com/presentation/d/151tR2hycoM3WqC9-7s9laMcCTZ9StY2fVLTpA_FwYBs/edit#slide=id.g2e736e3f9ff_3_94) 
- Please keep these slides up to copy/paste from as needed throughout the 
tutorial 
- Please copy/paste rather than typing everything word for word! 
  - Trick - Triple click to select a full code block for copy/paste! 
- [NRLMMD-GEOIPS/template_basic_plugin workshop-2024-beginner-solutions](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2024-beginner-solutions) 
- If all else fails, you can access the solutions on github.com 

## System requirements

- **CPU:** 1 CPU
- **RAM:** 40GB as the notebook is written, but could use more if modified.
  Reading the entire full-disk ABI image can take up to 100GB.
- **Disk Space:** 10GB storage space.

### Installing Appropriate Test Datasets

For this tutorial, you'll need GeoIPS test datasets that can be used to produce imagery 
or scientific datasets. 

If just attending the beginner tutorial, you'll need:

  - `test_data_clavrx`

These datasets might take a while to download so go ahead and download the appropriate 
datasets based on the following commands,.

In [ ]:
%%bash

# This is needed for both the beginner and advanced tutorial
geoips config install test_data_clavrx --outdir $GEOIPS_TESTDATA_DIR

### Introduction to GeoIPS

GeoIPS is a plugin-based system for processing geolocated data: 
  - produce imagery in several formats (most often PNG). 
  - produce output data products in NetCDF4 format. 
  - extended to add other output formats via plugins. 

![1111𝝁 infrared imagery](./images/conus_infrared.png)

![Himawari-9 CLAVR-x Cloud-Top-Height](./images/ahi_cloud_top_height.png)

![ABI CLAVR-x Cloud Top Height](./images/abi_cloud_top_height.png)

GeoIPS is almost entirely composed of plugins:

  - GeoIPS can be extended by developing new plugins in external python packages. 
  - No need to edit the main GeoIPS code to add new functionality. 
  - Most types of functionality in GeoIPS can be extended (and if something can’t be 
  extended, and you think it should, let us know!). 


### Vocabulary

**YAML**
  - "Yet Another Markdown Language" 
  - A human-readable data serialization language that is often used for writing configuration files 

**Plugin**
  - A Python module or YAML file that defines GeoIPS functionality 
  - Stored in an installable Python package that registers its plugin payload with GeoIPS 

**Interface**
  - A class of Python plugins that modify the same type of functionality within GeoIPS 
  (e.g., “the algorithms interface” or “the colormappers interface”) 
  
**Family**
  - A subset of an interface whose plugins accept different sets of arguments/properties 
  - Planning to deprecate this 'artifact' in the near future.


### Commonly used GeoIPS Plugin Types 

**algorithm**
  - Implements a function that modifies data and outputs new data

**colormapper**
  - Defines a method of applying a colormap to imagery

**feature_annotator**
  - Defines how to plot map features (e.g. coastlines, borders, rivers, etc.)

**gridline_annotator**
  - Defines how to plot gridlines and labels

**interpolator**
  - Defines a method of interpolating data to a sector

**output_formatter**
  - Defines a method for plotting imagery or outputting a data file 

**procflow**
  - Defines the order of operations to use when producing a product from GeoIPS 

**product**
  - Defines how to produce a specific product as a combination of other plugins
  - Uses other plugins (e.g. algorithm, colormapper, interpolator)

**product_defaults**
  - Defines a default set of plugins and arguments for producing a specific type of product 
  - Can be reused across multiple products to allow consistency between similar products 

**reader**
  - Defines a data reader 

**sector**
  - Describes an domain for reprojection of data 

## Hands On: Modify a plugin template to create your own installable plugin package  


### Get the Template Repository

To add new plugins to GeoIPS, we use Plugin Packages. You will use a template repository to build your own, installable plugin package containing custom plugins.

The following commands will clone the template plugin repository. Typically, when staring a new plugin package, you would choose the name of your package. We've taken that joy from you and have named your new package "cool_plugins". This name is stored in `$MY_PKG_NAME` for later use. 

`$MY_PKG_DIR` is also provided for convenience and contains the full path to your new package.



In [ ]:
%%bash

cd $GEOIPS_PACKAGES_DIR

if [[ ! -d "template_basic_plugin" && ! -d "$MY_PKG_NAME" ]]; then
    echo "Cloning template_basic_plugin package"
    git clone --no-tags --single-branch $GEOIPS_REPO_URL/template_basic_plugin.git
else
    echo "Package already exists"
fi

# Rename your package
if [[ ! -d "$MY_PKG_NAME" ]]; then
    echo "Renaming template_basic_plugin to $MY_PKG_NAME"
    mv -v template_basic_plugin/ $MY_PKG_NAME
else
    echo "Package already renamed"
fi

# This will remove references to our upstream repository for safety's sake
cd $MY_PKG_DIR
git remote remove origin 2> /dev/null || true

### Update the Package Name

Now, we can use `ls` to look around in your package directory.


In [ ]:
%%bash
cd $MY_PKG_DIR

ls --color

This package is set up to be an installable Python package named "my_package". Let's update it to install as "cool_plugins" instead.

Rename the default plugin package directory to "cool_plugins", then call `tree` to see the entire directory structure.

In [ ]:
%%bash
cd $MY_PKG_DIR

# Rename the package directory
if [[ -d "my_package" ]]; then
    echo "Moving my_package to $MY_PKG_NAME"
    git mv my_package $MY_PKG_NAME
else
    echo "Already moved"
fi

# Show the directory tree two-levels deep
tree -L 2

In [ ]:
%%bash

cd $MY_PKG_DIR/cool_plugins
tree

![plugins directory structure](./images/plugins_directory_structure.png)

### Update Pertinent Files

1. Update README.md (`vim README.md`)
  - Find/replace all occurrences of @package@  with your package name 
  - Vim Tip :%s/@package@/cool_plugins/g 
  
**Note**: The @ symbols are for ease of searching, take them out when you put your 
package name in!

2. Update pyproject.toml (`vim pyproject.toml`, more on this in soon.)
  - Find/replace all occurrences of my_package  with your package name 

3. Add and commit your changes.

#### Note

Due to the nature of Jupyter Notebooks, we are not able to edit files in place (that is <br />
opening them up and manually editing files (such as YAML or TOML)). Instead we'll <br />
update these files with Python. Keep in mind if you were running this notebook locally <br />
via an IDE, you'd be able to edit these files manually rather than using Python.

[Edit me](../../cool_plugins/README.md)

In [ ]:
"""Overwrite cool_plugins' pyproject.toml and README.md with correct contents."""

import os

with open("./updated_files/pyproject.toml", "r") as rf:
    toml_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/pyproject.toml", "w") as wf:
    wf.writelines(toml_lines)

with open("./updated_files/README.md", "r") as rf:
    md_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/README.md", "w") as wf:
    wf.writelines(md_lines)

In [ ]:
%%bash

cd $MY_PKG_DIR
git add README.md pyproject.toml
git commit -m "Updated name of template plugin package to mine" || true

#### Install your package 
Now that your package is updated, you can install it! 

We'll use `pip install -e $MY_PKG_DIR` where `-e` means "editable". This installs the package in "editable" mode so we can edit the package after it is installed and changes will be reflected in the installed package.

*For those who are interested, this acts similarly to a symlink in Linux, but has some more complexity behind it.*

In [ ]:
%%bash

pip install -e $MY_PKG_DIR

### See what you just installed 

Use the GeoIPS CLI to list the installed packages. Yours should be there!

In [ ]:
%%bash

geoips list packages

### A bit about pyproject.toml

Installing Python packages requires metadata that describes the package and how to 
install it. 

`pyproject.toml` defines this information for pip, including: 
  - Package name, version, description, license, etc. 
  - Which files should be contained in the package when installed 
  - How to build the package 

We make GeoIPS aware of our package using the `geoips.plugin_packages` namespace.
This allows GeoIPS to find all plugins within packages registered to this namespace.

GeoIPS automatically identifies all plugins defined within a plugin package via a plugin
registry. You can manually create these files via `geoips config create-registries`, 
however, GeoIPS will automatically create these files if a requested plugin cannot be
found. This usually occurs the first time GeoIPS is initialized.

**NOTE** for plugin registries to write successfully: 

1. All installed  plugin names within a given interface must be unique 

2. All installed plugins must be formatted and defined correctly 

We will make use of this more later! Modify plugin template solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

```
[tool.poetry.plugins."geoips.plugin_packages"]
"cool_plugins" = "cool_plugins" 
``` 

## GeoIPS Command Line Interface (CLI) Tutorial

The GeoIPS CLI can provide you with a lot of information about the installed plugin packages and their plugins. Please follow this [Jupyter Notebook](./CLI_Tutorial.ipynb) for instructions on how to 
make use of the GeoIPS CLI.

## Hands on: Create a Product Plugin (products  YAML-based interface) 

Solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data

Let's start by copying an existing product plugin to a new file. We'll modify it to create a new plugin.

In [ ]:
%%bash
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products
cp -v amsr2_using_product_defaults.yaml my_clavrx_products.yaml

### GeoIPS Yaml-based plugin properties

All YAML plugins will begin with these same four properties as shown below.

![Product top level keys](./images/product_top_level_keys.png)

### Edit my_clavrx_products.yaml properties

Update `my_clavrx_products.yaml` to match the following four lines *(Feel free to remove all lines preceded by `# @`)*:

```yaml
interface: products
family: list
name: my_clavrx_products
docstring: |
  CLAVR-x imagery products
```

Click to edit in a new tab: [my_clavrx_products.yaml](../../cool_plugins/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml)

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data 

Next, let's add our first new product! We'll start by adding a Cloud Top Height product based on CLAVR-x cloud property data. To do so, update the existing product as shown below.

![Update product spec](./images/update_product_spec.png)

Edit the file (**click me:** [my_clavrx_products.yaml](../../cool_plugins/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml)) again to update the product to match the following lines:

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud Top Height
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"]
```

### Update the Plugin Registry

Let's use the CLI to get more information about the plugin we just created.

While this won't be needed in the future (there is a bug...) we need to first rebuild the plugin registry to allow GeoIPS to locate your new plugin.

In [ ]:
!geoips config create-registry

### List the plugins

To ensure that your new plugin was installed and registered, you can call `geoips list products` or `geoips ls products`. With just that, you will list all products from all packages, though. To see only the plugins from the `cool_plugins` package, we add the `-p cool_plugins` option.

You should see your new plugin in the output below!

In [ ]:
!geoips ls products -p cool_plugins

### Describing plugins

To get more information about a particular plugin (or interface), call `geoips describe`. This is the generic format to follow via the CLI:

`geoips describe <interface_name> <source_name>.<plugin_name>`

In [ ]:
!geoips describe product clavrx.My-Cloud-Top-Height

#### `family: null`

You may have noticed that your product’s family was `null` in the output above. This isn’t the actual case. 

Products are the only plugin that may depend on another type of plugin. Products that depend on a `ProductDefaults` plugin will inherit their contents unless explicitly overridden. The family of the derived product_default is inherited by the product plugin.

For now, an easy way to check the family of your product is by describing its product 
default plugin (see above for its name).

*`ProductDefaults` plugins are discussed more later in this tutorial.*

In [ ]:
!geoips desc pdef Cloud-Height

### Use your new product!

To use the plugin you just created, we'll make use of the `geoips run` command.

- GeoIPS is called via a command line interface (CLI), as we've shown previously.
- The main command that you will use is `geoips run single_source`, which will run your 
  data through the specified procflow using the specified plugins 
- It's easiest to do this via a script, and scripts are stored in your plugin package's 
  `tests/` directory because they can be used later to regression test your package 
- Since we're running this in a notebook, we can easily run these commands with a code cell!

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --sector_list conus

### Viewing the log output

This will write some log output.  If your script succeeded it will end with 
`INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS` and open the file shown there (or run the cell below).

If successful, the output image should look like this:

![CLAVR-x CONUS My-Cloud-Top-Height](./images/clavrx-conus-my-cloud-top-height.png)

In [ ]:
from IPython.display import Image

Image(f"{os.environ['GEOIPS_OUTDIRS']}/preprocessed/annotated_imagery/NorthAmerica-UnitedStates-Continental/x-x-x/My-Cloud-Top-Height/clavrx/20230411.160020.goes-16.clavrx.My-Cloud-Top-Height.conus.52p06.cira.3p0.png")

### A word about Product Defaults 

GeoIPS has a number of product_defaults plugins defined to help you not reinvent the 
wheel. If specified in a Product, the values specified by a ProductDefaults plugin will
be added to the Product. **but**
- You can override any of the values from the ProductDefaults by re-specifying them in your Product.
- You can absolutely fully define a product without the use of a ProductDefaults plugin.

#### When to use Product Defaults
If you have product definition parameters that you want to reuse (i.e. if you're 
copy/pasting products), consider creating a ProductDefaults plugin.


This link will take you to the [product defaults defined by CLAVR-x plugin package](https://github.com/NRLMMD-GEOIPS/geoips_clavrx/tree/main/geoips_clavrx/plugins/yaml/product_defaults).

This link will take you to the [product defaults defined by the core GeoIPS package](https://github.com/NRLMMD-GEOIPS/geoips/tree/main/geoips/plugins/yaml/product_defaults).

The image below depicts the Cloud Height product defaults used by the CLAVR-x package. It defines:
- What interpolator to use on the data.
- How to call the `single_channel` algorithm for cloud height data (e.g. cloud top height or cloud base height).
- What colormap to apply to cloud height data and the data range it should be applied over.


![Cloud-Height Product Defaults](./images/Cloud-Height-product_defaults.png)

### Using ProductDefaults verbatim 

In your product you can use the product_defaults verbatim simply by specifying which product defaults to use.

![Product using product default](./images/product_using_product_default.png)

### Using ProductDefaults, but overriding some values 

You can also override just some parts of the product_defaults. 

In this example, we override the `algorithm` plugin contained in the Cloud-Height 
product_defaults, with our own specification. 

![Product default overridden](./images/product_default_overridden.png)

### Explicitly defining a Product (no ProductDefaults)

We also have the option to define a product without using product_defaults.

To do this: 
  - remove the `product_defaults` property 
  - add the `family` property

![Product fully specified](./images/product_fully_specified.png)

## Hands on: Add Additional Products to Your Product Plugin 


### Add Cloud Base Height (CBH) to the CLAVR-x Product Definition

- Using your definition of `My-Cloud-Top-Height` as an example, create a product 
  definition for `My-Cloud-Base-Height`

**Helpful Hints:** 
- The relevant variable in the CLAVR-x output file (and the equivalent GeoIPS reader) 
  is called `cld_height_base`
- The Cloud-Height product_default  can be used to simplify this product 
  definition (or you can DIY or override if you'd like!)

#### Edit me: [my_clavrx_products.yaml](../../cool_plugins/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml) (Close and save the file when done)

The block below will update the plugin registry and list the plugins from `cool_plugins`. Your new plugins should appear here!

In [ ]:
%%bash

geoips config create-registry
geoips list products -p cool_plugins

### Run the Cloud-Base-Height product

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Base-Height \
  --output_formatter imagery_annotated \
  --sector_list conus

In [ ]:
Image(f"{os.environ['GEOIPS_OUTDIRS']}/preprocessed/annotated_imagery/NorthAmerica-UnitedStates-Continental/x-x-x/My-Cloud-Base-Height/clavrx/20230411.160020.goes-16.clavrx.My-Cloud-Base-Height.conus.51p94.cira.3p0.png")

### Add Cloud Depth to the CLAVR-x Product Definition 

Using your definitions of `My-Cloud-Top-Height` and `My-Cloud-Base-Height` as examples, 
create a product definition for `My-Cloud-Depth`

**Helpful Hints:** 
We will define Cloud Depth for this tutorial as the difference between CTH and CBH 

**Note:**
This is meant to challenge you a bit! Give it a try and we'll go over the solution in 
the next markdown block

#### Edit me: [my_clavrx_products.yaml](../../cool_plugins/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml) (Close and save the file when done)

### Solution 

After editing, the `spec` portion of `my_clavrx_products.yaml` should now look like below. 

```yaml
spec:
  products: 
    - name: My-Cloud-Top-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    - name: My-Cloud-Depth
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud my Cloud Depth
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "cld_height_base", "latitude", "longitude"]
```

### A Problem
Take another look at the Product we just implemented, though. Note that it requires **two** non-geolocation variables, `cld_height_acha` and `cld_height_base`. Also, notice that we are using the `single_channel` algorithm. This is an algorithm that manipulates a single variable.

**We need a new Algorithm plugin!**

### Validate your new product plugins

Before you go ahead creating your new algorithm plugin, let's validate that the product
plugins we just created are found by GeoIPS. After running the block below, you should see the registry update, then each of the new Products descriptions.

In [ ]:
%%bash

geoips config create-registries
geoips describe product clavrx.My-Cloud-Top-Height
geoips describe product clavrx.My-Cloud-Base-Height
geoips describe product clavrx.My-Cloud-Depth

### Command: `geoips list`

You can also list all of the Products implemented by `cool_plugins` by listing them.

In [ ]:
%%bash

geoips list products -p cool_plugins

## Hands on: Create an Algorithm Plugin

Let's create our new Algorithm plugin!

First, let's copy an existing algorithm plugin to a new file named "my_cloud_depth.py" so we can use it as a template.

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/algorithms
cp -v pmw_89test.py my_cloud_depth.py

### The same three top-level attributes
Just like the Yaml-based plugins, this Module-based plugin has three top-level attributes that are common to all GeoIPS plugins: interface, family, and name. Here:
- `interface` is "algorithms" to indicate that this plugin belongs to the Algorithms interface.
- `family` is "xarray_to_xarray", a common family for algorithms indicating that it accepts an Xarray DataSet as input and returns an Xarray DataSet as output.
- `name` is the name of the algorithm (which we will update).

![Updating top level portions of new algorithm](./images/updating_algorithm_top_level.png)

### Let's update our `my_cloud_depth` algorithm with the following:

```python
"""Cloud depth product.

Difference of cloud top height and cloud base height.
"""

import logging

from xarray import DataArray

LOG = logging.getLogger(__name__)

interface = "algorithms"
family = "xarray_to_xarray"
name = "my_cloud_depth"
```

#### Edit me: [my_cloud_depth.py](../../cool_plugins/cool_plugins/plugins/modules/algorithms/my_cloud_depth.py) (Close and save the file when done)

### Updating your Algorithm

All module-based plugins, including Algorithms, must include a `call()` function. This function is what is called when the plugin is executed. The `call()` function's signature is determined by the algorithm's family.

Our cloud height algorithms have an addional `"scale_factor"` parameter in their call signatures that the algorithm we copied does not.
To update this algorithm, replace the signature of the `call()` function and its docstring with the following. 

*You can remove the comments if desired.*

```python
def call(
    xobj,  # Xarray Dataset holding DataArrays
    variables, # list of required input variables for the algorithm. These are ordered as specified in your product plugin.
    product_name,
    output_data_range,
    scale_factor,  # Adding a scale factor here for use in converting input meters to output kilometers
    min_outbounds="crop",
    max_outbounds="mask",
    norm=False,
    inverse=False,
):
    """My cloud depth product algorithm manipulation steps."""
```

#### Edit me: [my_cloud_depth.py](../../cool_plugins/cool_plugins/plugins/modules/algorithms/my_cloud_depth.py) (Close and save the file when done)

### Update the call function's functionality

Inside the `call()` is where the actual data manipulation occurs. To update the algorithm to produce cloud depth, replace the contents of the `call()` function with the following. If you examine the code below, you will see that it:
- Extracts the relevant variables from the xarray object.
- Subtracts cloud base from cloud top height.
- Normalizes over the specified data range.
- Packages the data as an Xarray DataArray.
- Adds the DataArray to the DataSet using the product name as the key.
- Returns the resulting Xarray DataSet.

The returned DataSet will contain all of the original input data as well as the newly created data.

**Hint: Make sure to index the variable list to the order of the variables you defined in your product.**

```python
    # Variables in the order defined in your Cloud-Depth Plugin
    cth = xobj[variables[0]]
    cbh = xobj[variables[1]]

    out = (cth - cbh) * scale_factor

    from geoips.data_manipulations.corrections import apply_data_range

    # Data manipulation: Anything you want!
    data = apply_data_range(
        out,
        min_val=output_data_range[0],
        max_val=output_data_range[1],
        min_outbounds=min_outbounds,
        max_outbounds=max_outbounds,
        norm=norm,
        inverse=inverse,
    )
    xobj[product_name] = DataArray(data)

    return xobj
```

#### Edit me: [my_cloud_depth.py](../../cool_plugins/cool_plugins/plugins/modules/algorithms/my_cloud_depth.py) (Close and save the file when done)

Once complete, let's describe the plugin you just created to make sure it registered appropriately.

In [ ]:
%%bash

# This is a bug-fix. I am not sure why this gets created, but let's just delete it.
rm $HOME/cool_plugins/cool_plugins/plugins/modules/algorithms/.ipynb_checkpoints/my_cloud_depth-checkpoint.py

geoips config create-registries
# In case you forgot the name of your plugin, you can run:
geoips ls algs -p cool_plugins
# Now let's describe that plugin
geoips describe algorithm my_cloud_depth

## Overriding the My-Cloud-Depth Algorithm

As mentioned earlier, we can override a product's defaults by adding new information to <br />
that product's `spec` object. 

This was complete when we previously added our `My-Cloud-Depth` product, however it's <br />
important to understand how this functionality works. Since that product inherits from <br />
the `Cloud-Height` product_defaults plugin, `My-Cloud-Depth` assumes it would use the <br />
`single_channel` algorithm. We want to override that with the algorithm we just created. <br />

The following YAML code demonstrates how you can override a product_default plugin in <br />
a product plugin.

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    # Update the code block below
    - name: My-Cloud-Depth 
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Cloud my Cloud Depth 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_acha", "cld_height_base", "latitude", "longitude"] 
        # Algorithm override portion
        algorithm: 
          plugin:
            name: my_cloud_depth # The name we assigned our algorithm when we defined it
            arguments: 
              output_data_range:  [0, 20]
              scale_factor:  0.001
```

#### Edit me: [my_clavrx_products.yaml](../../cool_plugins/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml) (Close and save the file when done)

### Hands on: Using Your Algorithm Plugin

Run the following command to make use of your new algorithm plugin, referenced in our <br />
`My-Cloud-Depth` product.

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

geoips run single_source \
  $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter imagery_annotated \
  --sector_list conus

In [ ]:
Image(f"{os.environ['GEOIPS_OUTDIRS']}/preprocessed/annotated_imagery/NorthAmerica-UnitedStates-Continental/x-x-x/My-Cloud-Depth/clavrx/20230411.160020.goes-16.clavrx.My-Cloud-Depth.conus.51p94.cira.3p0.png")

## Part 2 -- Tutorial Contents

We're going to show you how to extend GeoIPS by adding new plugins for five new interfaces. 

We will add:  
- One new sector 
- One new feature_annotator , one new gridline_annotator 
- Three new colormappers 
- One new output_formatter

We will also show you how to: 
- Produce a NetCDF4 file using your algorithm output 
- Read your custom NetCDF4 file back in, and produce imagery from it 

### Hands on: Outputting your new product as NetCDF 

The following code block produces your `My-Cloud-Depth` product using the `netcdf_geoips`<br />
output formatter plugin. Instead of an image, this command will produce a NetCDF file <br />
using the same data as the previous image.


In [ ]:
%%bash

geoips run single_source \
   $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
   --reader_name clavrx_hdf4 \
   --product_name My-Cloud-Depth \
   --output_formatter netcdf_geoips \
   --filename_formatter geoips_netcdf_fname \
   --sector_list conus

There will output a bunch of log output. If your script succeeded it will end with <br />
`INTERACTIVE: Return Value 0`. To view your output, look for a line that says <br />
`SINGLESOURCESUCCESS`.

Open the NetCDF file to view its contents (e.g. with Panoply; shown below)<br />
![My Cloud Depth NetCDF Output](./images/panoply-netcdf-output.png)

**Note** - For more information on the output_formatter 
you just used, run:  

In [ ]:
%%bash

geoips describe output-formatter netcdf_geoips

### Sectors

Sectors are YAML plugins that define an area of interest for plotting in GeoIPS.

**Two types:** 
- **static**: static sector of a specific area 
- **dynamic**: sectors that are generated at run-time to follow an event (e.g. 
  tropical cyclone, atmospheric river, pyro-Cb, etc.) 

For this tutorial, we will be creating a **static**  sector. 

**Static Sector Properties:** 
- **Metadata**: Contains information related to the origin of the sector. This includes 
  continent, country, state, etc.
- **Projection**: Proj projection information 
  - **a**: The radius of the earth (in meters) 
  - **lat_0**: Center latitude coordinate 
  - **lon_0**: Center longitude coordinate 
  - **proj**: String representing projection type 
  - **units**: String representing units of the 
    projection 
- **Resolution**: The size of each pixel in meters 
- **Shape**: Image shape in pixels (width + height) 
- **Center**: The center x/y coordinates of the sector 

### Example Sectors

![Example Sectors](./images/example_sectors.png)

You can run the following commands to get more information about each of the sectors 
shown here by running:

In [ ]:
%%bash

geoips describe sector australia
geoips describe sector central_america
geoips describe sector south_pole

### Generating Static Sector Properties 

To generate the properties necessary to create a static sector, you must first define <br /> 
the area you want to display (captured by the appropriate satellite), based on center <br /> 
latitude and longitude bounds. 

The easiest way to do this is to open a [Google Maps](https://www.google.com/maps/) tab, <br /> 
then right click the center of your sector to obtain the center latitude and longitude <br /> 
values. 

Once you have those center latitude and longitude values, you’re ready to create your <br />
custom sector. 

The next slide will display the changes you will need to make to create a custom conus <br />
sector plugin. 

![Google Maps CONUS](./images/google_maps_conus.png)

**Note** - For listing of sector plugins, run: 

In [ ]:
%%bash

geoips list sectors

### Hands on: Creating Custom Sectors 

Let's start by copying a sector file to edit

In [ ]:
%%bash

# Make a sectors folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static

# Copy a sector file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp -v "$geoips_path/plugins/yaml/sectors/static/australia.yaml" my_conus_sector.yaml

### Generating Static Sector Properties 

The following YAML code demonstrates how to implement your own custom sector. <br />
Since we're running this in a notebook, we'll update this file for you automatically, <br />
via the following **code block**.

You'll notice that the metadata remains untouched, however it's important to note that <br />
this section is very helpful for displaying additional information about the sector, not <br />
only for the backend of GeoIPS, but also for people using this sector plugin.

```yaml
interface: sectors
family: area_definition_static 
name: my_conus_sector 
docstring: "My CONUS Sector" 
metadata: 
  region: 
    continent: NorthAmerica 
    country: UnitedStates 
    area: x
    subarea: x
    state: x
    city: x
spec:
  area_id: my_conus_sector 
  description: CONUS
  projection: 
    a: 6371228.0 
    lat_0: 37.0
    lon_0: -96.0
    proj: eqc
    units: m
  resolution: 
    - 3000
    - 3000
  shape: 
    height: 1000
    width: 2200
  center: [0, 0]
```


In [ ]:
"""Update my_conus_sector with the correct contents."""

import os
import yaml

with open("./updated_files/my_conus_sector.yaml", "r") as yaml_file:
    updated_sector = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/sectors/static/my_conus_sector.yaml", "w") as f:
    yaml.safe_dump(updated_sector, f, default_flow_style=False, sort_keys=False)

### Testing Your Custom Static Sector 

The commands you ran in the previous slide create a custom conus sector. <br />
`my_conus_sector.yaml` will be an  example plugin, showing you that you can create sectors <br />
just like `conus.yaml`, to your own specifications. 

To quickly check whether or not you like the shape and resolution of your custom sector, <br />
you can use the command line function create_sector_image . This will plot and save <br />
images containing the borders and coastlines of the inputted sectors. For example, to test <br />
your custom sector, run the following: 

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

geoips config create-registries
geoips test sector my_conus_sector

Once completed, open the `my_conus_sector.png`  image to see what your sector will look like. <br />
It should look like this:

![My CONUS Sector](./images/my_conus_sector.png)

**Note** -  You can describe the sector plugin you just 
created by running:  

In [ ]:
%%bash

geoips describe sector my_conus_sector

### Using Your Custom Static Sector

To use `my_conus_sector.yaml` in your test script, simply replace `--sector_list conus` <br />
with `--sector_list my_conus_sector`.

Let's run a command that now uses your new sector plugin.


Let's run that script you just created and modified.

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --sector_list my_conus_sector

The result of executing that script should look like this:

![CTH my conus sector](./images/my_conus_sector_cloud_top_height.png)

## Hands on: Creating Custom Gridline/Feature Annotators 

### Defining gridline_annotators and feature_annotators 

**gridline_annotators:**
- Describe the format of the grid lines shown in your imagery. Can control: 
- **Labels**, and which to display 
- **Lines**, such as their **color**, **linestyle**, and **linewidth** 
- **Spacing**, such as the distance between **latitude** and **longitude** labels, and 
  the grid lines that represent them. 

**feature_annotators:**
- Describe the format of the features shown in your imagery. Can control: 
- **Coastlines**, **Borders**, **States**, and **Rivers**. All of these features have 
  the same parameters: 
  - Whether they are **enabled** 
  - The **color** of the feature 
  - The **linewidth** of the feature displayed 

### Example Gridline and Feature Annotators 

![Example gridline and feature annotators](./images/example_gridline_and_feature_annotators.png)

You can describe the aforementioned plugins via:

In [ ]:
%%bash

geoips describe feature-annotator default
geoips describe gridline-annotator default

### Creating a Custom gridline_annotator 

Let's begin by setting up a directory for your gridline annotators and copy over an 
existing plugin from GeoIPS.

In [ ]:
%%bash

# Make a gridline_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators

# Copy a gridline_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp -v "$geoips_path/plugins/yaml/gridline_annotators/default.yaml" tutorial.yaml

### Breaking down a gridline_annotator

The following image explains each component of a gridline_annotator plugin.

![Gridline Annotator Explained](./images/gridline_annotator_explained.png)

Now that you understand what a gridline_annotator plugin does, we'll modify the plugin we <br />
copied over. 

Since we're running in a notebook, we'll automatically do this for you. However, please <br />
take in the contents of `tutorial.yaml` with the YAML shown below.

```yaml
interface: gridline_annotators 
family: cartopy
name: tutorial 
docstring: |
  The tutorial gridline_annotators configuration.  
  All gridline labels enabled, latitude and 
  Longitude lines colored mediumseagreen, 2.5 degree 
  spacing, 1px linewidth, and [5, 3] linestyle. 
spec:
  labels:  
    top: true
    bottom: true
    left: true
    right: true
  lines:  
    color: mediumseagreen 
    linestyle: [5, 3]
    linewidth: 1
  spacing:  
    latitude: 2.5
    longitude: 2.5
```

In [ ]:
"""Update gridline_annotator - tutorial.yaml with the correct contents."""

import os

with open("./updated_files/gridline_annotators/tutorial.yaml", "r") as yaml_file:
    updated_gridline_annotator = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/gridline_annotators/tutorial.yaml", "w") as f:
    yaml.safe_dump(updated_gridline_annotator, f, default_flow_style=False, sort_keys=False)

### Creating custom feature_annotator 

Now that we've created a custom gridline_annotator plugin, let's do the same for a <br />
feature_annotator plugin.

In [ ]:
%%bash

# Make a feature_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators

# Copy a feature_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp -v "$geoips_path/plugins/yaml/feature_annotators/default.yaml" tutorial.yaml

### Breaking down a feature_annotator

The following image explains how a feature is set up in a feature_annotator plugin.

You'll notice that a feature can either be disabled, and if not, you must define its <br /> 
color and linewidth.

![Feature Annotator Explained](./images/feature_annotator_explained.png)

Now that you understand what a feature_annotator plugin does, we'll modify the plugin we <br />
copied over. 

Since we're running in a notebook, we'll automatically do this for you. However, please <br />
take in the contents of `tutorial.yaml` with the YAML shown below.


```yaml
interface: feature_annotators
family: cartopy
name: tutorial 
docstring: |
  The tutorial feature annotators configuration.  
  All line types enabled. All colored  
  [midnightblue, red, yellow, cyan]. 2px coastlines, 
  2px borders, 2px states, 1px rivers .
spec:
  coastline:  
    enabled: true
    edgecolor : midnightblue 
    linewidth : 2
  borders:  
    enabled: true
    edgecolor: red
    linewidth: 2
  states:  
    enabled: true
    edgecolor: darkslategray 
    linewidth: 2
  rivers:  
    enabled: true
    edgecolor: cyan
    linewidth: 1
```

In [ ]:
"""Update feature_annotators - tutorial.yaml with the correct contents."""

import os

with open("./updated_files/feature_annotators/tutorial.yaml", "r") as yaml_file:
    updated_feature_annotator = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/feature_annotators/tutorial.yaml", "w") as f:
    yaml.safe_dump(updated_feature_annotator, f, default_flow_style=False, sort_keys=False)

### Using Your Custom Annotators

To add the effects of these plugins to your final output, simply add the following lines <br /> 
to your command that uses the `"imagery_annotated”` output formatter.

```bash
--feature_annotator tutorial \
--gridline_annotator tutorial \
```

Once you've added those, you can run the script via:

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

# build the new plugin registries
geoips config create-registries

geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter imagery_annotated \
  --feature_annotator tutorial \
  --gridline_annotator tutorial \
  --sector_list conus

If you copied the YAML for the gridline and feature -annotators verbatim, the result of <br />
that script should look like this:

![Feature / Gridline annotator image](./images/feature_gridline_annotator.png)

## Colorbars and Colormaps Background and Introduction 


### Using a Matplotlib Colormap 

Matplotlib has many predefined colormaps.

The `matplotlib_linear_norm` colormapper plugin is capable of using these predefined <br />
colormaps - simply provide the name of the colormap in the `“cmap_name”` argument when <br />
you define your product plugin.

For now `matplotlib_linear_norm` is the only plugin that can directly use named <br />
colormaps (which explicitly normalizes the colors using matplotlib’s `“Normalize”` <br />
function), though others may be added in the future with additional methods of <br />
manipulating the color scales.

The image below displays a sample product plugin using matplotlib built-in colormap.

![Product using matplotlib_linear_norm](./images/product_using_mpl_linear_norm.png)

### Using an ASCII Colormap 

The `matplotlib_linear_norm` plugin can also leverage `ASCII` colormap files installed <br />
within GeoIPS, installed within a plugin package, or stored in an arbitrary location on disk. 

To use an ascii colormap, specify:
  - `cmap_source: ascii` 
  - `cmap_name: <cmap_name>`

  
Installed ASCII colormaps are stored within a plugin package under `plugins/txt/ascii_palettes/`.
  - The filename must be of the form `<cmap_name>.txt` (this is how geoips finds the given <br />
    `cmap_name`)
  - The colors are defined as RGB triplets ranging from 0 to 256 and should be formatted <br /> 
    as three columns of 256 integers. 
  - Commented lines are allowed (prefaced by “#”), to provide additional context relating <br />
    to the physical meaning of the colormap (ie, min and max expected values/units, <br />
    transition points, etc) 
  - For an example, see the [tpw_purple](https://github.com/NRLMMD-GEOIPS/geoips/blob/710b58ae82f479168e7da49dfccd8650730478d8/geoips/plugins/txt/ascii_palettes/tpw_purple.txt) colormap in the geoips package. 

If you would like to specify an arbitrary full path on disk rather than installing your <br />
ascii palette within your  plugin package, additionally specify cmap_path . This is most <br />
useful for research, development, and testing purposes. 
  - `cmap_path: <full_path>/<any_name>.txt`

`matplotlib_linear_norm` provides a number of options that can be used to customize how <br />
the colormap is used and how the colorbar is drawn. 

The following image displays a sample product using ascii palette -based colormap.

![Product using ascii colormap](./images/product_using_ascii_colormap.png)

### Using a custom GeoIPS Python-based colormapper

Color information can also be specified via a python-based GeoIPS `“colormapper”` plugin, <br />
allowing customization using specific matplotlib commands and utilities. <br />
This is the most flexible method.

The following images walk you through each part of a colormapper plugin.

![Colormapper top level](./images/colormapper_top_level.png)

![Colormapper inside call function](./images/colormapper_inside_call_function.png)

The `“mpl_colors_info”` return dictionary is what GeoIPS uses within the matplotlib-based <br />
utilities and output formatters in order to ensure consistent application of colors, in <br />
both the imagery and the colorbars. 

Coming up, we'll walk you through creating both GeoIPS Python-based colormapper, as well <br />
as ASCII-palette based matplotlib_linear_norm products.

![Colormapper mpl_colors_info](./images/mpl_colors_info.png)

## Hands on: Creating your own custom Python-based colormapper 

Let's begin by creating a directory to hold our colormappers and copying over an existing <br />
colormapper from GeoIPS.

In [ ]:
%%bash

# Make a colormappers folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers

# Create an __init__.py file
touch __init__.py

# Copy a colormapper file from GeoIPS
geoips_path="$(pip show geoips_clavrx | awk '/^Location: / {print $2 "/geoips_clavrx"}')"
cp -v "$geoips_path/plugins/modules/colormappers/cmap_cldHeight.py" colorful_cloud_height.py

Now, lets update the file we copied over with new content. We'll replace the upper <br />
portions of `colorful_cloud_height.py` with the following python code.

Note: Since we're using a notebook, this will be done for you in the following code block.

```python
"""Module containing colormap for colorful cloud height product.""" 

import logging

LOG = logging.getLogger(__name__) 

interface = "colormappers" 
family = "matplotlib" 
name = "colorful_cloud_height" 


def call(data_range=[ 0, 20]):
    """Colorful cloud height colormap."""
```

Once that's complete, let's update the contents of the `call()` function to change the <br />
colormap we'll produce. Feel free to choose different colors if you'ld like.

In this case, we only modify the colors, but you just as easily could modify the transition <br />
values as well. These are all just direct matplotlib/python commands creating the desired <br />
color information.

```python
    transition_vals = [
        (min_val, 1),
        (1, 2),
        (2, 3),
        (3, 4),
        (4, 6),
        (6, 8),
        (8, 10),
        (10, 15),
        (15, max_val),
    ]
    transition_colors = [
        ("pink", "red"),
        ("paleturquoise", "teal"),
        ("plum", "rebeccapurple"),
        ("yellow", "chartreuse"),
        ("limegreen", "darkgreen"),
        ("wheat", "darkorange"),
        ("darkgray", "black"),
        ("lightgray", "silver"),
        ("lightskyblue", "deepskyblue"),
    ]
```

In [ ]:
"""Replace the contents of colorful_cloud_height.py with the correct colormapper information."""

import os

with open("./updated_files/colorful_cloud_height.py", "r") as rf:
    python_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/modules/colormappers/colorful_cloud_height.py", "w") as wf:
    wf.writelines(python_lines)

## Hands on: Using your new python-based colormapper in a new product

### Add custom colormapper to new product. 

Let's begin by creating a new `Cloud-Base-Python-Colors` product, using `Cloud-Height` <br />
as the product_defaults.

We'll do this automatically in the next code cell.

Normally, you'd just add the following product entry to your `my_clavrx_products.yaml` <br />
file. Note that it makes use of the colormapper we just put together!

```yaml
    - name: Cloud-Base-Python-Colors
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Colorful Cloud Base Height, 
        Using a python-based custom colormapper. 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_base", "latitude", "longitude"] 
        colormapper: 
          plugin: 
            name: colorful_cloud_height 
            arguments: {}
```

In [ ]:
"""Add My-Cloud-Depth to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]
my_cloud_depth = updated_products["spec"]["products"][2]
cloud_base_python_colors = updated_products["spec"]["products"][3]

updated_products["spec"]["products"] = [
    my_cloud_top_height,
    my_cloud_base_height,
    my_cloud_depth,
    cloud_base_python_colors,
]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

Let's modify the following line of our previous product command with the following.

```bash
--product_name My-Cloud-Top-Height \
```

with

```bash
--product_name Cloud-Base-Python-Colors \
```

Now, you can produce your new product which uses the colormapper we created!

In [ ]:
%%bash

export CARTOPY_DATA_DIR=$HOME/cartopy

# rebuild your registry files
geoips config create-registries
geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name Cloud-Base-Python-Colors \
  --output_formatter imagery_annotated \
  --sector_list conus

You should get the following image from that script:

![Colorful Cloud Height](./images/colorful_cloud_height.png)